### Projeto de Automação para Verificação de Processos de Licenciamento Ambiental no Sistema SEI!MG

#### Descrição do Projeto

- Este projeto tem como objetivo facilitar a verificação e monitoramento de processos de licenciamento ambiental no sistema SEI!MG, automatizando a coleta de informações sobre atualizações e respostas dos órgãos ambientais de forma dinâmica.

#### Licença

- Este projeto é distribuído sob a [licença MIT](LICENSE), o que significa que você pode usá-lo, modificá-lo e distribuí-lo livremente, desde que mantenha a atribuição de direitos autorais.

------------------------------

1. [Dúvidas? Veja esse vídeo de explanação sobre o programa](https://youtu.be/Dj8KMo_B60E)

2. [Meu perfil do GitHub contendo este e outros projetos](https://github.com/feltbeats)

3. [Link para compartilhamento e execução do projeto](https://colab.research.google.com/drive/1TLOG_BpfDr7Xj0-ce8hcSSt6Xt2d-IIL?usp=sharing)

In [ ]:
!pip install webdriver_manager
!pip install selenium
!apt-get update
!apt-get -q upgrade -y
!apt-get install chromium-browser
!apt-get -q install -y chromium-chromedriver
!pip install seleniumbase
!pip install easygui
!pip install PyQt5
!pip install tk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [555 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,009 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,131 kB]
Get:10 http

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 23.4 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import getpass
import shutil
import easygui
from PyQt5.QtWidgets import QFileDialog, QApplication
import sys
from tkinter import Tk, filedialog

In [ ]:
# Configuração do webdriver
options = webdriver.ChromeOptions()
options.binary_location = '/usr/bin/chromium-browser'
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')

navegador = webdriver.Chrome(options=options)

In [ ]:
def fazer_login():
    # Passo 1 - selecionar o site
    url = "https://www.sei.mg.gov.br/sei/controlador_externo.php?acao=usuario_externo_logar&id_orgao_acesso_externo=0"
    navegador.get(url)

    # Pedir ao usuário para inserir o nome de usuário
    usuario = input("Digite o seu usuário: ")

    # Pedir ao usuário para inserir a senha de forma segura (oculta)
    senha = getpass.getpass("Digite a sua senha: ")

    # Preencher campos de usuário e senha
    navegador.find_element(By.XPATH, '//*[@id="txtEmail"]').send_keys(usuario)
    navegador.find_element(By.XPATH, '//*[@id="pwdSenha"]').send_keys(senha)

    # Passo 3 - clicar no botão e entrar
    navegador.find_element(By.XPATH, '//*[@id="sbmLogin"]').click()

    try:
        # Verificar se o login foi bem-sucedido
        if navegador.find_element(By.ID, 'divInfraBarraLocalizacao').text == 'Controle de Acessos Externos':
            print("Login bem-sucedido!")
        else:
            print("Login falhou. Verifique seu usuário e senha e tente novamente.")
            fazer_login()  # Solicitar que o usuário insira novamente as credenciais
    except NoSuchElementException:
        print("Login falhou. Verifique seu usuário e senha e tente novamente.")
        fazer_login()  # Solicitar que o usuário insira novamente as credenciais

# Iniciar o processo de login
fazer_login()

Digite o seu usuário: d.tavaresassis@gmail.com
Digite a sua senha: ··········
Login bem-sucedido!


In [ ]:
# Inicialização de listas para armazenar URLs de processos
urls_processos = []

# Loop para coletar URLs de processos
while True:
    elementos_links_processos = navegador.find_elements(By.XPATH, '//*[@id="tblDocumentos"]//td[2]/a')

    if not elementos_links_processos:
        break

    urls_processos += [elemento.get_attribute('href') for elemento in elementos_links_processos]

    try:
        paginacao = navegador.find_element(By.ID, 'lnkInfraProximaPaginaSuperior')
        paginacao.click()
        time.sleep(3)
    except NoSuchElementException:
        break

# Inicialização de lista para armazenar informações dos processos
informacoes = []

# Loop para coletar informações dos processos
for url_processo in urls_processos:
    try:
        navegador.get(url_processo)

        numero_processo_pagina = navegador.find_element(By.XPATH, '//*[@id="tblCabecalho"]/tbody/tr[2]/td[2]').text
        tabela_cabecalho = navegador.find_element(By.ID, 'tblCabecalho')
        dado1 = tabela_cabecalho.find_element(By.XPATH, '//*[@id="tblCabecalho"]/tbody/tr[3]/td[2]').text
        dado2 = tabela_cabecalho.find_element(By.XPATH, '//*[@id="tblCabecalho"]/tbody/tr[4]/td[2]').text
        tabela_documentos = navegador.find_element(By.ID, 'tblDocumentos')
        elementos_ultima_linha = tabela_documentos.find_elements(By.XPATH, '//*[@id="tblDocumentos"]/tbody/tr[last()]/td')
        dado3 = elementos_ultima_linha[2].text
        dado4 = elementos_ultima_linha[3].text
        dado5 = elementos_ultima_linha[4].text

        informacoes.append({'Número de Processo Página': numero_processo_pagina, 'Tipo de Processo': dado1, 'Data de Criação': dado2, 'Último Documento': dado3, 'Data de Última Movimentação': dado4, 'Unidade do Documento': dado5})
    except Exception as e:
        print(f"Erro ao acessar a URL: {url_processo}")
        print(f"Erro: {e}")
        continue  # Pula para o próximo link em caso de erro

# Criação de um DataFrame com as informações
df = pd.DataFrame(informacoes, columns=['Número de Processo Página', 'Tipo de Processo', 'Data de Criação', 'Último Documento', 'Data de Última Movimentação', 'Unidade do Documento'])

# Nome do arquivo de saída
nome_arquivo = "dados_processos.xlsx"

# Salvar o DataFrame em um arquivo Excel
df.to_excel(nome_arquivo, index=False)


Erro ao acessar a URL: javascript:void(0);
Erro: Message: unknown error: unsupported protocol
  (Session info: headless chrome=118.0.5993.70)
Stacktrace:
#0 0x595366ff7fb3 <unknown>
#1 0x595366ccb4a7 <unknown>
#2 0x595366ccdaa4 <unknown>
#3 0x595366d4e3b5 <unknown>
#4 0x595366d34942 <unknown>
#5 0x595366d4dc02 <unknown>
#6 0x595366d34713 <unknown>
#7 0x595366d0718b <unknown>
#8 0x595366d07f7e <unknown>
#9 0x595366fbd8d8 <unknown>
#10 0x595366fc1800 <unknown>
#11 0x595366fcbcfc <unknown>
#12 0x595366fc2418 <unknown>
#13 0x595366f8f42f <unknown>
#14 0x595366fe64e8 <unknown>
#15 0x595366fe66b4 <unknown>
#16 0x595366ff7143 <unknown>
#17 0x7cad16989ac3 <unknown>



In [ ]:
from google.colab import files

# Mova o arquivo para o diretório /content, caso ainda não esteja lá
import shutil
shutil.move(nome_arquivo, '/content/' + nome_arquivo)

# Gere um link para o usuário baixar o arquivo
files.download(nome_arquivo)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>